In [1]:
from sentence_transformers import SentenceTransformer
import pandas as pd
from chromadb.utils import embedding_functions
from tqdm import tqdm
import chromadb
from chromadb.config import Settings
device = 'cuda'
model = SentenceTransformer("intfloat/multilingual-e5-small")

def make_passage(ans):
    return f'passage: {ans}'

def make_query(q):
    return f'query: {q}'

def make_db(df_path, db_name):
    df = pd.read_excel(df_path)

    documents = []
    embeddings = []
    metadatas = []
    ids = []

    for index, row in tqdm(df.iterrows(), total=len(df)):
        documents.append(row['Answer'])
        embedding = model.encode(sentences=make_passage(row['Answer']), batch_size=1, device = device).tolist()
        embeddings.append(embedding)
        metadatas.append({'source': row['Theme']})
        ids.append(str(index + 1))

    client = chromadb.Client(Settings(persist_directory='db/'))
    invest_qa_emb = client.create_collection(name=db_name, metadata={'hnsw:space': 'cosine'})

    invest_qa_emb.add(
        documents=documents,
        embeddings=embeddings,
        metadatas=metadatas,
        ids=ids
    )

c:\Users\Ирина\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df_path = 'data\data.xlsx'
db_name = "invest_qa_emb"
make_db(df_path=df_path, db_name=db_name)

100%|██████████| 1412/1412 [00:16<00:00, 87.48it/s]


In [2]:
from sentence_transformers import SentenceTransformer
import pandas as pd
from chromadb.utils import embedding_functions
from tqdm import tqdm
import chromadb
from chromadb.config import Settings

client = chromadb.Client()
# invest_qa_emb = client.create_collection(name=db_name, metadata={'hnsw:space': 'cosine'})
collection = client.create_collection(name="my_collection", embedding_function=embedding_functions.SentenceTransformerEmbeddingFunction(model_name="intfloat/multilingual-e5-small"), metadata={'hnsw:space': 'cosine'})


'c:\\Users\\Ирина\\Desktop\\RAG\\develop'

In [3]:
from sentence_transformers import SentenceTransformer
import pandas as pd
from chromadb.utils import embedding_functions
from tqdm import tqdm
import chromadb
from chromadb.config import Settings

client = chromadb.Client()
# invest_qa_emb = client.create_collection(name=db_name, metadata={'hnsw:space': 'cosine'})
collection = client.create_collection(name="my_collection", embedding_function=embedding_functions.SentenceTransformerEmbeddingFunction(model_name="intfloat/multilingual-e5-small"), metadata={'hnsw:space': 'cosine'})

df = pd.read_excel('data/data.xlsx')
collection.add(
    documents=df.Answer.to_list(),
    metadatas=[{'question': q} for q in df.Theme.to_list()],
    ids=[str(i) for i in range(len(df))]
)

embeddddd = SentenceTransformer("intfloat/multilingual-e5-small")
query = "Что делать Если пришел маржин-колл"
input_em = embeddddd.encode(sentences=f"query: {query}", batch_size=1, device = 'cuda').tolist()

results = collection.query(
    query_embeddings=[input_em],
    n_results=5
)

In [5]:
embeddddd = SentenceTransformer("intfloat/multilingual-e5-small")
query = "Что делать Если пришел маржин-колл"
input_em = embeddddd.encode(sentences=f"query: {query}", batch_size=1, device = 'cuda').tolist()

results = collection.query(
    query_embeddings=[input_em],
    n_results=5
)